#Visulastion of the missile alerts in Israel in May-June
caused mainly by Hamas terror organization in Gaza strip.
The following code was (and still is) written by Yarden Levenberg.

In [ ]:
!pip install geopy
!pip install geopandas

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 515, in _det

In [ ]:
from google.colab import files
uploaded = files.upload()
# please import the file "alerts_may_june.txt"

# if you have the alerts excel, you can upload it in the next code line, otherwise proceed as usual:
#alerts_excel = files.upload()
## please import the file "alerts_data.xlsx"
# alerts = pd.read_excel(alerts_excel)

Saving alerts_may_june.txt to alerts_may_june (3).txt


# Data sorting & organizing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('alerts_may_june.txt', delimiter = "\r" ,  names = ['date','time', 'location'], skiprows=4)
data = {"DateTime":np.array(df.iloc[0::3,0])+" "+np.array(df.iloc[1::3,0]), "location": np.array(df.iloc[2::3,0])}
alerts = pd.DataFrame.from_dict(data)
alerts['DateTime'] = pd.to_datetime(alerts['DateTime'])
display(alerts)

,DateTime,location
0,2021-06-18 00:17:00,כפר עזה
1,2021-05-21 01:49:00,מבטחים עמיעוז ישע
2,2021-05-21 01:49:00,צוחר ואוהד
3,2021-05-21 01:49:00,כיסופים
4,2021-05-21 01:34:00,סופה
...,...,...
7040,2021-09-05 21:11:00,נתיב העשרה
7041,2021-09-05 21:11:00,אזור תעשייה הדרומי אשקלון
7042,2021-09-05 21:11:00,זיקים
7043,2021-09-05 21:11:00,יד מרדכי


# Locating coordinates for locations using geopy

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="name_of_your_app") # geolactor works only when given a string as app name

locating Gaza city coordinates as source for missiles 
(although inaccurate, real location of missile launch locations is unavailable to the public)

In [ ]:
location = geolocator.geocode("עזה")
gaza_location = [location.latitude, location.longitude]
print(gaza_location)

[31.5128679, 34.4581358]


get coordinates of all uniqe places at the dataframe, find their coordinates and assign them back into the dataframe. if coordinates are outside Israel's coordinates "box", assiagned them to a 'fail_names' list for manual locating.

please note, this code box might take a good few minutes.

In [ ]:
location_list = np.array(df.iloc[2::3,0])
np.unique(location_list)
name_lat_lon = []
failed_names = []
for name in np.unique(location_list):
  geo_location = geolocator.geocode(str(name))
  if geo_location == None:
    failed_names.append(name)
  elif geo_location.latitude < 29.5 or geo_location.latitude > 33.2 or geo_location.longitude < 34.24 or geo_location.longitude> 35.9:
    failed_names.append(name)
  else:
    name_lat_lon.append((name,geo_location.latitude ,geo_location.longitude) )
df_loc_lat_lon = pd.DataFrame(name_lat_lon, columns=['location', 'latitude', 'longitude'])

In [ ]:
print("Failed names are:\n", failed_names)

Failed names are:
 ['אבו קרינאת והפזורה', 'אבו-תלול והפזורה', 'אום בטין והפזורה', 'אזור תעשייה אפק ולב הארץ', 'אזור תעשייה הדרומי אשקלון', 'אזור תעשייה כנות', 'אזור תעשייה נשר - רמלה', 'אזור תעשייה עידן הנגב', 'אשדוד - א,ב,ד,ה', 'אשדוד - ח,ט,י,יג,יד,טז', 'אשדוד - יא,יב,טו,יז,מרינה', 'בארי', 'בית סוהר השרון', 'בניה', 'בצרה', 'גבים, מכללת ספיר', 'גיאה', 'גפן', 'גת', "דריג'את, תל ערד ואל פורה", 'הרצליה - מרכז וגליל ים', 'זיתן', 'זמרת ושובה', 'חוות שיקמים', 'ירושלים - מזרח, מרכז ומערב', 'ירושלים - צפון ואלונים', 'כוחלה מכחול', 'כסייפה והפזורה', "כפר הרי''ף וצומת ראם", 'כפר מימון ותושיה', 'כפר נוער בן שמן', 'לקיה והפזורה', 'מבטחים עמיעוז ישע', 'מולדה, אל-סייד והפזורה', 'מטווח ניר עם', 'מעגלים גבעולים מלילות', 'משגב עם', 'מתחם בני דרום', 'מתחם פי גלילות', 'מתחם צומת שוקת', 'מתן', 'נווה', 'נתניה - מזרח', 'ענב', 'ערוער - ערערה בנגב', 'פארק תעשיות פלמחים', 'פארק תעשייה ראם', 'פטיש', 'פנימיית עין כרם', 'צוחר ואוהד', 'קצר-א-סיר והפזורה', 'קריית גת, כרמי גת', 'רהט והפזורה', 'רנן', 'שגב שלום והפזור

I have manually located the places in the 'failed_names' list and found a name that works for them with the Geolocator

In [ ]:
second_try = ['אבו קרינאת', 'אבו-תלול', 'אום בטין', 'Afek industrial park', "Ni'ilya", 'Bnei Ayish', 'מפעל נשר רמלה', 'פארק תעשיות עידן הנגב', 'בית חולים אסותא אשדוד', 'גן גולדה אשדוד', 'Beeri, israel', 'hasharon prison, israel', 'Bnaya, israel', 'Batzra, israel', 'מכללת ספיר', 'Gea, israel', 'Gefen, israel', 'Gat, israel', "תל ערד", 'קניון שבעת הכוכבים', 'Zeitan', 'זימרת', 'Havat Shikmim, israel', 'מוזיאון ישראל', 'הר חוצבים', 'כוחלה', 'כסייפה', 'כפר הריף', 'כפר מימון', 'Ben Shemen Youth Village, israel', 'לקיה', 'עמיעוז', 'Mulada, israel', 'Nir Am', 'גבעולים', 'Misgav Am', 'טרויה', 'מחלף גלילות מערב', 'תל שוקת', 'מושב מתן', 'מושב נווה', 'Netanya east', 'Einav, israel', 'ערערה בנגב', 'גן לאומי שפך נחל שורק', 'פארק תעשיות ראם', 'Patish, israel', 'Olsvanger Garden, israel', 'צוחר', 'קצר-א-סיר', 'קריית גת', 'רהט', 'רנן מרכז', 'שגב שלום שוקייב א סאלם', 'ת. רכבת ראש העין צפון', 'תל אביב פלורנטין', 'Dizengoff square, tel aviv', 'Nahshonim, israel', 'Tkuma, israel', 'תראבין א-צאנע']
failed_names_again = []
for i in range(len(second_try)):
  name = failed_names[i]
  geo_location = geolocator.geocode(str(second_try[i]))
  if geo_location == None:
    failed_names_again.append(name)
  elif geo_location.latitude < 29.5 or geo_location.latitude > 33.3 or geo_location.longitude < 34.24 or geo_location.longitude> 35.9:
    failed_names_again.append(name)
  else:
    name_lat_lon.append((name,geo_location.latitude ,geo_location.longitude) )

# print success if all names were fixed
if failed_names_again == []:
  print("SUCCEES")
else:
  print("FAIL, CHECK THE FOLLOWING:\n")
  failed_names_again

SUCCEES


In [ ]:
natural_numbers = [1,2,3,4,5,6,7]
even_numbers = filter(natural_numbers, lambda x : x%2==0 )


twice_large_array = map(natural_numbers, lambda x : 2*x)

condition = lambda x : x > 5

filter(natural_numbers, condition)

twice = [2*i for i in natural_numbers]

# merge alerts (datetime and location) with df_loc_lat_lon to a new and complete dataframe - "alerts"

In [ ]:
alerts = pd.merge(alerts, df_loc_lat_lon, on="location")
display(alerts)

,DateTime,location,latitude,longitude
0,2021-06-18 00:17:00,כפר עזה,31.483335,34.533322
1,2021-05-21 01:12:00,כפר עזה,31.483335,34.533322
2,2021-05-21 00:28:00,כפר עזה,31.483335,34.533322
3,2021-05-20 21:00:00,כפר עזה,31.483335,34.533322
4,2021-05-20 20:16:00,כפר עזה,31.483335,34.533322
...,...,...,...,...
5849,2021-10-05 18:02:00,בית נקופה,31.802857,35.125269
5850,2021-10-05 18:02:00,צובה,31.784654,35.125955
5851,2021-10-05 18:02:00,עין נקובא,31.793619,35.119127
5852,2021-10-05 18:02:00,מוצא עילית,31.793494,35.156563


alerts dataframe is now complete, in order to save time you can now save it to excel and next time use it immediately.

In [ ]:
# alerts.to_excel('alerts_data.xlsx')
# files.download('alerts_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# The visulastion (interactive plot)
currently showing all the data points on the map

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# fig = px.scatter_geo(alerts, lon=alerts.longitude, lat=alerts.latitude, hover_name="location") #, animation_frame="DateTime"
# fig.update_geos(resolution=50, showcountries=True, countrycolor="Black", showsubunits=True, subunitcolor="grey") #, fitbounds="locations")
# fig.show()

working version (no animation)

In [ ]:
fig = go.Figure()

origin_lat = [gaza_location[0] for i in range(len(alerts.latitude))]
origin_lon = [gaza_location[1] for i in range(len(alerts.longitude))]

source_to_dest = zip(origin_lat, alerts.latitude , origin_lon, alerts.longitude, alerts.location)


fig = px.scatter_geo(alerts, lon=alerts.longitude, lat=alerts.latitude, hover_name=alerts.location)

## Loop thorugh each missile entry to add line between source and destination
for slat, dlat, slon, dlon, d_name in source_to_dest:
    fig.add_trace(go.Scattergeo(lat = [slat,dlat], lon = [slon, dlon], name=d_name, mode = 'lines'))

fig.update_geos(resolution=50, showcountries=True, countrycolor="Black", showsubunits=True, subunitcolor="grey")

fig.update_layout(showlegend=False) #,title_text = 'Connection Map Depicting missiles from Gaza to Israel',
#                   height=700, width=900,
#                   margin={"t":0,"b":0,"l":0, "r":0, "pad":0})

fig.show()

testing site:

(now working on animation)

In [ ]:
fig = go.Figure()

origin_lat = [gaza_location[0] for i in range(len(alerts.latitude))]
origin_lon = [gaza_location[1] for i in range(len(alerts.longitude))]

source_to_dest = zip(origin_lat, alerts.latitude , origin_lon, alerts.longitude, alerts.location)


fig = px.scatter_geo(alerts, lon=alerts.longitude, lat=alerts.latitude, hover_name=alerts.location)

## Loop thorugh each missile entry to add line between source and destination
for slat, dlat, slon, dlon, d_name in source_to_dest:
    fig.add_trace(go.Scattergeo(lat = [slat,dlat], lon = [slon, dlon], name=d_name, mode = 'lines')) #) #, animation_frame="DateTime"))

fig.update_geos(resolution=50, showcountries=True, countrycolor="Black", showsubunits=True, subunitcolor="grey")

fig.update_layout(showlegend=False)

fig.show()

TypeError: ignored